In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm

from benford_helper_functions import normalize, get_first_digit, benfords_test

In [ ]:
p = 1
low, high = p * np.log(10), (p+1) * np.log(10)

x_ = np.random.uniform(low=low, high=high, size=100000)
x = np.exp(x_)

np.unique(get_first_digit(x), return_counts=True)

In [ ]:
from stat_tests import chi2_test, ks_test

In [ ]:
import sys

sys.path.append('../..')
from plotting.matplotlib_setup import configure_latex, savefig, set_size_decorator, savefig, thiner_border

tex_dir, images_dir = 'porocilo/main.tex', 'porocilo/images'

configure_latex(style=['science', 'notebook'], global_save_path=images_dir)

%config InlineBackend.figure_format = 'pdf'

In [ ]:
def get_lognorm_pdf(x, mu, sigma):
    return (1 / (x * sigma * np.sqrt(2 * np.pi))) * np.exp(-((np.log10(x) - mu)**2) / (2*sigma**2))

def lognormMu(x, mu, s):
    tempX = x / np.exp(mu)
    return lognorm.pdf(tempX, s)

In [ ]:
from scipy.stats import lognorm


def lognorm_params(mode, stddev):
    """
    Given the mode and std. dev. of the log-normal distribution, this function
    returns the shape and scale parameters for scipy's parameterization of the
    distribution.
    
    https://stackoverflow.com/questions/41464753/generate-random-numbers-from-lognormal-distribution-in-python
    """
    p = np.poly1d([1, -1, 0, 0, -(stddev/mode)**2])
    r = p.roots
    sol = r[(r.imag == 0) & (r.real > 0)].real
    shape = np.sqrt(np.log(sol))
    scale = mode * sol
    return shape, scale


def lognorm_params_exact(mode, stddev):
    a = stddev**2 / mode**2
    x = 1/4*np.sqrt(-(16*(2/3)**(1/3)*a)/(np.sqrt(3)*np.sqrt(256*a**3+27*a**2)-9*a)**(1/3) +
                    2*(2/3)**(2/3)*(np.sqrt(3)*np.sqrt(256*a**3+27*a**2)-9*a)**(1/3)+1) + \
        1/2*np.sqrt((4*(2/3)**(1/3)*a)/(np.sqrt(3)*np.sqrt(256*a**3+27*a**2)-9*a)**(1/3) -
                    (np.sqrt(3)*np.sqrt(256*a**3+27*a**2)-9*a)**(1/3)/(2**(1/3)*3**(2/3)) +
                    1/(2*np.sqrt(-(16*(2/3)**(1/3)*a)/(np.sqrt(3)*np.sqrt(256*a**3+27*a**2)-9*a)**(1/3) +
                                 2*(2/3)**(2/3)*(np.sqrt(3)*np.sqrt(256*a**3+27*a**2)-9*a)**(1/3)+1))+1/2) + \
        1/4
    shape = np.sqrt(np.log(x))
    scale = mode * x
    return shape, scale


N = 10**6
mu = 10**3

# sigmas = np.array([100, 1000, 10000, 100000]) # -> [0.5, 0.8, 1.2]
sigmas = np.logspace(2, 12, 100)

lognorm_dists, lognorm_sigmas = [], []
for s in sigmas:
    sigma, scale = lognorm_params(mu, s)
    lognorm_sigmas.append(sigma[0])
    
    np.random.seed(1)
    lognorm_rng = lognorm.rvs(sigma, 0, scale, size=N)
    
    lognorm_dists.append(lognorm_rng)

In [ ]:
f1s = []

for i, lognorm in enumerate(lognorm_dists):
    bins = np.logspace(np.floor(np.log10(lognorm.min())), 
                       np.floor(np.log10(lognorm.max())) + 1, 
                       len(lognorm))
    
    pdf, _ = np.histogram(lognorm, bins=bins)
    
    bins = bins[:-1]
    bins = np.log10(bins)
    pdf = normalize(pdf, bins)
    
    f1 = benfords_test(pdf, bins)
    f1s.append(f1)
    
    first_digits = get_first_digit(lognorm)
    _, n = np.unique(first_digits, return_counts=True)
    n = n / np.sum(n)
    
    print(f'delez 1: {n[0]:.4f}, f1: {f1:.3e}, normal sigma: {sigmas[i]:.2f}, lognorm sigma: {lognorm_sigmas[i]:.2f}')

(Uniform distribution characterization). A sequence of real numbers (respectively, a Borel measurable function, a random variable, a Borel probability measure) is Benford if and only if the decimal logarithm of its absolute
value is uniformly distributed modulo 1.

In [ ]:
sigmas = np.sqrt(np.log10(1 + sigmas / mu**2))

In [ ]:
lognorm_fracs = []

for dist in lognorm_dists:
    # lognorm_fracs.append(get_number_fracs_math(np.log10(dist[dist > 0])))
    lognorm_fracs.append(np.log10(dist) % 1)

In [ ]:
lognorm_fracs_hists = []

for i, fracs in enumerate(lognorm_fracs):
    n, bins, _ = plt.hist(fracs, bins=30, density=False, histtype='step')
    lognorm_fracs_hists.append([n, bins[1:]])
    
    plt.close()

# Test $\chi^2$

In [ ]:
# chi2_test(np.array(lognorm_fracs), n_bins=30)

In [ ]:
from scipy.stats import chisquare
from scipy.stats import chi2

In [ ]:
chi2_test = []

N, n_bins = np.sum(lognorm_fracs_hists[0][0]), len(lognorm_fracs_hists[0][1])

for hist in lognorm_fracs_hists:
    chi2_ = chisquare(hist[0], f_exp=N/n_bins)
    chi2_test.append([chi2_.statistic, chi2_.pvalue])

In [ ]:
for c in chi2_test:
    print(f'{c[0]:.3f}, {c[1]:.3f}')

In [ ]:
alpha = 0.01 # stopnja pomembnosti

#                         stopnja zaupanja
critical_value_chi2 = chi2.ppf(1 - alpha, n_bins)
critical_value_chi2

# Test Kolmogorov-Smirnova

In [ ]:
ks_test(np.array(lognorm_fracs))

In [ ]:
from scipy.stats import kstest
from scipy.stats import kstwo

In [ ]:
ks_test = []

for dist in lognorm_fracs:
    ks = kstest(dist, cdf='uniform', alternative='two-sided', args=(0, 1))
    stat, p = ks.statistic, ks.pvalue
    ks_test.append([stat, p])
    print(f'{stat:.2e}, {p:.3f}')

In [ ]:
critical_value_ks = kstwo.ppf(1 - alpha, len(lognorm_fracs[0]))
f'{critical_value_ks:.2e}'

In [ ]:
if False:
    fig, axs = set_size_decorator(plt.subplots, fraction=1, ratio='4:3')(2, 2)
    axs = axs.flatten()

    for i, fracs in enumerate(lognorm_fracs):
        axs[i].hist(fracs, bins=30, density=False, histtype='step')
        lognorm_fracs_hists.append([n, bins[1:]])

        # plt.close()

        axs[i].ticklabel_format(style='sci', axis='y', scilimits=(0, 0))

        an1 = f'$\sigma$ = {sigmas[i]:.2f}, $\sigma_X$ = {lognorm_sigmas[i]:.2f}'
        an2 = f'\n$\chi^2$ = {chi2_test[i][0]:.2f}, $d$ = {ks_test[i][0]:.2e}'
        an3 = f'\n$\chi^2_*$ = {critical_value_chi2:.2f}, $d_*$ = {critical_value_ks:.2e}'
        an4 = r' pri $\alpha$ = {}'.format(alpha)

        if i == 3:
            an = an1 + an2 + an3 + an4
        else:
            an = an1 + an2

        axs[i].annotate(an, xy=(0.1, 0.1), xycoords='axes fraction', fontsize=8)

    savefig('lognorm_uniform_hists')

In [ ]:
fig, ax = set_size_decorator(plt.subplots, fraction=0.5, ratio='4:3')(1, 1)

ax.set_yscale('log')

x = lognorm_sigmas

y = [i[0] for i in chi2_test]
ax.scatter(x, y, s=5)
ax.plot(x, y, lw=1)

ax.set_xlabel('$\sigma_X$')

ax.set_ylabel('$\chi^2$', c='C0')
ax.tick_params(axis='y', labelcolor='C0')

ax.set_ylim([0.9e1, 1e7])

ax2 = ax.twinx()

ax2.set_yscale('log')
ax2.minorticks_off()
ax2 = thiner_border(ax2)

y2 = [i[0] for i in ks_test]
ax2.scatter(x, y2, s=5, c='C2')
ax2.plot(x, y2, lw=1, c='C2')

ax2.set_ylabel('$d$', c='C2')
ax2.tick_params(axis='y', labelcolor='C2')

x3 = sigmas

ax3 = ax.twiny()
ax3 = thiner_border(ax3)
# ax3.set_xscale('log')

ax3.plot(x3, y, lw=0)

ax3.set_xlabel('$\sigma$')

ax.grid(zorder=0, alpha=0.5)

# savefig('stat_lognorm_tests', tight_layout=False)

In [ ]:
chi2_lst = np.array([i[0] for i in chi2_test])
ks_lst = np.array([i[0] for i in ks_test])

In [ ]:
f1s = np.array(f1s)

In [ ]:
idx = np.argsort(f1s)

fig, ax = set_size_decorator(plt.subplots, fraction=0.5, ratio='4:3')(1, 1)

ax.set_xscale('log')
ax.set_yscale('log')
ax.scatter(f1s[idx], chi2_lst[idx], s=3, label='$\chi^2$')
ax.scatter(f1s[idx], ks_lst[idx], s=3, label='KS', c='C2')

ax.set_ylabel('test')
ax.set_xlabel('$f_1$')

ax.legend()

# savefig('test_lognorm_f1_chi2_KS')